In [3]:
#!pip install boto3

# Step 1: Initialize the S3 client
import boto3
import json
import os
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

s3 = boto3.client('s3')


In [4]:
# Step 2: Retrieve AWS Account ID to verify credentials
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
print(f"Currently authenticated AWS Account ID: {account_id}")


Currently authenticated AWS Account ID: 684115525159


In [5]:
# Step 3: Define upload parameters
bucket_name = 'your-bucket-name'  # Replace with your S3 bucket name - Use the bucket name, not the ARN. Why don't we use the ARN?
file_name = 'index.html'  # The name of the HTML file to upload
file_path = '/path/to/index.html'  # The local path to your HTML file


In [6]:
# Step 4: Validate the file path
if not os.path.isfile(file_path):
    raise FileNotFoundError(f"The file at {file_path} does not exist.")

In [11]:
# Step 5: Upload the file to the S3 bucket
try:
    response = s3.upload_file(
        Filename=file_path, 
        Bucket=bucket_name, 
        Key=file_name,
        ExtraArgs={
            'ContentType': 'text/html',  # Ensure the correct MIME type
        }
    )
    print("File uploaded successfully!")
except NoCredentialsError:
    print("Error: AWS credentials not found.")
except PartialCredentialsError:
    print("Error: Incomplete AWS credentials configuration.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File uploaded successfully!


In [8]:
# Step 6: Retrieve the public URL of the uploaded file
object_url = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
print(f"Your file is available at: {object_url}")

Your file is available at: https://ppol5206.s3.amazonaws.com/index.html


In [7]:
# Step 7: Update the bucket policy to allow public access
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{bucket_name}/*"

        }
    ]
}
try:
    s3.put_bucket_policy(
        Bucket=bucket_name,
        Policy=json.dumps(bucket_policy)
    )
    print("Bucket policy updated to allow public read access.")
except Exception as e:
    print(f"Error updating bucket policy: {e}")

Bucket policy updated to allow public read access.
